In [27]:
from google_bigquery import bq_to_csv, csv_to_bq
import pandas as pd
import json
import yaml
import re
import glob
from datetime import date
import unidecode
import numpy as np
import string
from keyword import iskeyword

In [28]:
dataset_id = 'maxi_prices'
table_id ='price'
folder = str(date.today())
store = '/maxi/'
path = '/Users/simonprudhomme/Desktop/maxi/'
path+folder+store

'/Users/simonprudhomme/Desktop/maxi/2020-04-01/maxi/'

In [29]:
all_files = glob.glob(path+folder+store + "/*.csv")

li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, sort=True,ignore_index=True)
print(data.shape)

(7536, 5)


In [30]:
def clean_attributs(attribut):
    s = attribut
    try:
        json_acceptable_string = s.replace("'", "\"")
        d = json.loads(json_acceptable_string)[0]
    except:
        d = yaml.load(s,Loader=yaml.FullLoader)[0]
    return d

data['attributs'] = [clean_attributs(attribut) for attribut in data['attributs']]

In [31]:
data_clean = pd.DataFrame()
for item in data['attributs']:
    sub_data = pd.DataFrame.from_dict(item,orient='index')
    data_clean = pd.concat([data_clean,sub_data],axis=1)
data_clean = data_clean.T.reset_index(drop=True)

data_clean['ratio'] = data['ratios']
data_clean['sales'] = data['sales']
data_clean['productCategory'] = data['name']
data_clean['date'] = data['date']

data_clean = data_clean.drop_duplicates().reset_index(drop=True)

In [32]:
def remove_ponctuation(x):
    if x is not np.nan:
        x = unidecode.unidecode(x)
        #x = x.translate(str.maketrans('', '', string.punctuation))
        x = x.replace(';\n;','')
        #x = x.strip()
    else:
        x = np.nan
    return x

data_clean['productName'] = [remove_ponctuation(x) for x in data_clean['productName']]

In [33]:
def is_numeric(x):
    if x is not np.nan:
        if any(i.isdigit() for i in x):
            return True
    else:
        return False

In [34]:
def clean_sale(x):
    if ' for ' in str(x):
        x = np.nan
    if ' MIN ' in str(x):
        x = np.nan
    if x is not np.nan :
        x = x.split('$')[1]
        x = x.strip()
        x = float(x)
    else:
        x = np.nan
    return x

data_clean['productSales'] = [clean_sale(x) if is_numeric(x) is True else np.nan for x in data_clean['sales']]

In [35]:
data_clean['productPrice'] = data_clean['productPrice'].astype(float)
data_clean['productPricePerGram'] = data_clean['ratio'].str.split('/')
data_clean['productUnit'] = data_clean['ratio'].str.split('/')

def clean_price(x):
    if x is not np.NaN:
        if len(x)>1 :
            try:
                x = x[0].replace('$','')
                x = x.replace(' ','')
                x = x.replace(',','.')
                x = float(x)
            except:
                x = np.nan
    else:
        x
    return x

def clean_unit(x):
    if x is not np.NaN:
        if len(x)>1 :
            x = x[1].strip()
    else:
        x
    return x

data_clean['productPricePerGram'] = [clean_price(x) for x in data_clean['productPricePerGram']]
data_clean['productUnit'] = [clean_unit(x) for x in data_clean['productUnit']]

In [36]:
data_clean['productURL'] = 'https://www.maxi.ca/p/'+data_clean['productSKU']
data_clean['productSKU'] = data_clean['productSKU'].str.split('_')
data_clean['productSKU'] = [x[0] for x in data_clean['productSKU']]
data_clean['productSKU'] = data_clean['productSKU'].str.strip()
data_clean['productSKU']

0          21183409
1          21082062
2       20559141001
3       20839325002
4       20006357001
           ...     
5292       21204248
5293       20971249
5294       20044848
5295       20079455
5296       21204849
Name: productSKU, Length: 5297, dtype: object

In [37]:
data_clean.shape

(5297, 21)

In [38]:
data_clean = data_clean[['productSKU', 'productName', 'productBrand', 'productPrice', 'dealBadge', 
            'productSales', 'productPricePerGram', 'productUnit','productURL','productCategory','date']]
data_clean['source'] = 'maxi'

In [39]:
filename_clean = str(date.today())+'_clean.csv'
data_clean.to_csv(path+folder+'/'+filename_clean,index=False)